# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "sw"
target_language = "en" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/workspace/nlp/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/workspace/nlp/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [2]:
!echo $gdrive_path

/workspace/nlp/masakhane/sw-en-baseline


In [3]:
# Install opus-tools
! pip install opustools-pkg

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-sw.xml.gz not found. The following files are available for downloading:

        ./JW300_latest_xml_en.zip already exists
        ./JW300_latest_xml_sw.zip already exists
   8 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-sw.xml.gz

   8 MB Total size
./JW300_latest_xml_en-sw.xml.gz ... 100% of 8 MB
gzip: JW300_latest_xml_sw-en.xml.gz: No such file or directory


In [7]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["src"] = source_language 
os.environ["trg"] = target_language 


! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-01-21 13:24:05--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.2’

test.en-any.en.2    100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2021-01-21 13:24:06 (11.2 MB/s) - ‘test.en-any.en.2’ saved [277791/277791]

--2021-01-21 13:24:07--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-en.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-01-21 13:24:07 ERROR 404: Not Found.

mv: cannot stat 'test.

In [6]:
source_file = "jw300."+source_language
target_file = "jw300."+target_language

# They should both have the same length.
! wc -l "$source_file"
! wc -l "$target_file"
tok_source_file = source_file+".tok"
tok_target_file = target_file+".tok"

# Tokenize the source
! sacremoses -l "$source_language" tokenize < "$source_file" > "$tok_source_file"
# Tokenize the target
! sacremoses -l "$target_language" tokenize < "$target_file" > "$tok_target_file"

# Let's take a look what tokenization did to the text.
! head "$source_file"*
! head "$target_file"*

# Change the pointers to our files such that we continue to work with the tokenized data.
source_file = tok_source_file
target_file = tok_target_file

979527 jw300.sw
979527 jw300.en
==> jw300.sw <==
“ Tazama !
Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
Iliyopo juu ni ahadi itokayo kwa Mungu ambayo imechaguliwa iwe kichwa cha broshua yenye kuvuvia tumaini .
Ahadi hiyo yamaanisha nini kwetu ?
Itatimizwa jinsi gani , na wakati gani ?
Pata kujua kwa kusoma kichapo hiki cha kurasa 32 kilicho na picha za kupendeza . Kwa Kshs .
18.00 ( Tshs . 90.00 ; RWF 90 ) , utapelekewa broshua hiyo , pamoja na broshua Serikali Itakayoleta Paradiso na Furahia Milele Maisha Duniani !
Tafadhali mnipelekee zile broshua tatu “ Tazama !
Mimi Ninafanya Vitu Vyote Kuwa Vipya , ” Serikali Itakayoleta Paradiso , na Furahia Milele Maisha Duniani !
Mimi nawapelekea nyinyi Kshs .

==> jw300.sw.tok <==
“ Tazama !
Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
Iliyopo juu ni ahadi itokayo kwa Mungu ambayo imechaguliwa iwe kichwa cha broshua yenye kuvuvia tumaini .
Ahadi hiyo yamaanisha nini kwetu ?
Itatimizwa jinsi gani , na wakati gani ?
Pata kujua kwa kusoma kichapo hiki cha k

In [8]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 1276/979526 lines since contained in test set.


,source_sentence,target_sentence
0,“ Tazama !,“ Look !
1,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”,I Am Making All Things New ”
2,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo ...,The above is a promise from God that has been ...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [10]:
df.to_csv("en_sw_pairs.csv", index=False)

Todo: Add FB XNLI pairs - https://docs.google.com/spreadsheets/d/1C_-4F3d5eRFwT0prtt-gt_-5TH27q0g-WN6O0PJIbys/edit?usp=sharing

In [11]:
import pandas as pd
df = pd.read_csv("en_sw_pairs.csv")
len(df)


978251

In [12]:
df

,source_sentence,target_sentence
0,“ Tazama !,“ Look !
1,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”,I Am Making All Things New ”
2,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo ...,The above is a promise from God that has been ...
3,Ahadi hiyo yamaanisha nini kwetu ?,What does that promise mean for us ?
4,"Itatimizwa jinsi gani , na wakati gani ?","How will it be fulfilled , and when ?"
...,...,...
978246,Miongoni mwao ni wahubiri wa kizazi cha tatu c...,Among these publishers today are third - gener...
978247,Tunamshukuru Yehova na ndugu na dada hao waami...,We give thanks to Jehovah and to those early f...
978248,"15 : 15 , 16 . — Hifadhi yetu ya vitu vya kale...","15 : 15 , 16 . ​ — From our archives in Portug..."
978249,"Tazama makala yenye kichwa , “ Kuna Kazi Zaidi...",See “ There Is More Harvest Work to Be Done ” ...


In [13]:
df['source_sentence'] = df['source_sentence'].str.replace('\d+', '')
df['source_sentence'] = df['source_sentence'].str.replace('*', '')
df['source_sentence'] = df['source_sentence'].str.replace('!', '')
df['source_sentence'] = df['source_sentence'].str.replace(':', '')
df['source_sentence'] = df['source_sentence'].str.replace(';', '')
df['source_sentence'] = df['source_sentence'].str.replace('  ', '')
df['source_sentence'] = df['source_sentence'].str.rstrip('\ ')
df['source_sentence'] = df['source_sentence'].str.rstrip('\"')
df['source_sentence'] = df['source_sentence'].str.rstrip('\.')

df['target_sentence'] = df['target_sentence'].str.replace('\d+', '')
df['target_sentence'] = df['target_sentence'].str.replace('*', '')
df['target_sentence'] = df['target_sentence'].str.replace('!', '')
df['target_sentence'] = df['target_sentence'].str.replace(':', '')
df['target_sentence'] = df['target_sentence'].str.replace(';', '')
df['target_sentence'] = df['target_sentence'].str.replace('  ', '')
df['target_sentence'] = df['target_sentence'].str.rstrip('\ ')
df['target_sentence'] = df['target_sentence'].str.rstrip('\"')
df['target_sentence'] = df['target_sentence'].str.rstrip('\.')

In [14]:
df['length'] = df['source_sentence'].str.len()
df.sort_values('length', ascending=True, inplace=True)
len(df)

978251

In [15]:
df2 = df[df.length > 11]
len(df2)

930876

In [16]:
df2['length'] = df2['target_sentence'].str.len()
df2.sort_values('length', ascending=True, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
df3 = df2[df2.length > 11]
len(df3)

917728

In [18]:
df3

,source_sentence,target_sentence,length
729760,[ Maelezo ya Chini ],[ Footnote ],12.0
305784,Kupanda au Kuteremka ?,Up or Down ?,12.0
388177,Kunahusisha nini ?,What is it ?,12.0
298318,"Ndiyo , ni muhimu","Yes , it is",12.0
366227,"Márquez ) ,/","Várguez ) ,/",12.0
...,...,...,...
611738,“ Watekelezaji wa Neno la Mungu ” Mahali pa M...,“ Doers of God’s Word ” District Convention L...,16720.0
506639,Njooni Kwenye “ Wasifaji Wenye Shangwe ” Mkusa...,Come to the “ Joyful Praisers ” District Conve...,16919.0
527710,“ Wajumbe wa Amani ya Kimungu ” Mahali pa Mik...,“ Messengers of Godly Peace ” District Conven...,17066.0
106352,Karibuni Kwenye “ WAJUMBE WA AMANI YA KIMUNGU ...,Welcome to the “ Messengers of Godly Peace ” D...,17484.0


In [19]:
df3.to_csv('en_sw_pairs_ordered.csv',index=False)

In [20]:
import pandas as pd
df = pd.read_csv("en_sw_pairs_ordered.csv")
len(df)

917728

In [21]:
df

,source_sentence,target_sentence,length
0,[ Maelezo ya Chini ],[ Footnote ],12.0
1,Kupanda au Kuteremka ?,Up or Down ?,12.0
2,Kunahusisha nini ?,What is it ?,12.0
3,"Ndiyo , ni muhimu","Yes , it is",12.0
4,"Márquez ) ,/","Várguez ) ,/",12.0
...,...,...,...
917723,“ Watekelezaji wa Neno la Mungu ” Mahali pa M...,“ Doers of God’s Word ” District Convention L...,16720.0
917724,Njooni Kwenye “ Wasifaji Wenye Shangwe ” Mkusa...,Come to the “ Joyful Praisers ” District Conve...,16919.0
917725,“ Wajumbe wa Amani ya Kimungu ” Mahali pa Mik...,“ Messengers of Godly Peace ” District Conven...,17066.0
917726,Karibuni Kwenye “ WAJUMBE WA AMANI YA KIMUNGU ...,Welcome to the “ Messengers of Godly Peace ” D...,17484.0


In [22]:
df_pp = df

In [11]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144796 sha256=a672b4c48e25c303e96476a57daff44ea0e5e6d99efc41a4a1c494ca2eefcd39
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [23]:
# start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [23]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.bpe.en <==
 M@@ ad@@ s@@ en , F 
V@@ is@@ u@@ ali@@ ze the s@@ cen@@ e 
New As@@ sign@@ ments
 Corinthi@@ ans@@ ,
L@@ AN@@ D@@ S AN@@ D P@@ E@@ O@@ P@@ L@@ E@@ S
W@@ RI@@ T@@ E@@ R I@@ N R@@ U@@ S@@ S@@ I@@ A
No@@ ah . ​ — Lu@@ ke
() Bra@@ z , D 
Be Th@@ an@@ k@@ ful for the R@@ ain
Do Not Jud@@ ge One A@@ no@@ ther

==> train.bpe.sw <==
 M@@ ad@@ s@@ en , F 
W@@ az@@ ia kik@@ ao 
M@@ ig@@ a@@ wo M@@ ip@@ ya
 Wakorinth@@ o,
N@@ C@@ H@@ I N@@ A W@@ AT@@ U
N@@ C@@ H@@ IN@@ I U@@ R@@ U@@ S@@ I
N@@ oa . — L@@ uka
() Bra@@ z , D 
T@@ ham@@ ini M@@ v@@ ua
M@@ si@@ hu@@ ku@@ mi@@ ane

==> train.en <==
[ Footnote ]
Up or Down ?
What is it ?
Yes , it is 
Várguez ) ,/
ADVERTISING 
 Heggli , T 
[ Footnote ]
P . , Canada
V . , France

==> train.sw <==
[ Maelezo ya Chini ]
Kupanda au Kuteremka ?
Kunahusisha nini ?
Ndiyo , ni muhimu 
Márquez ) ,/
MATANGAZO YA BIASHARA 
 Heggli , T 
[ Maelezo ya Chini ]
P . , Kanada
V . , Ufaransa
==> dev.bpe.en <==
During my@@ years of full - time service 



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [24]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip install .
# Install Pytorch with GPU support v1.7.1.
! pip install torch==1.7.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /workspace/masakhane-mt/joeynmt
  Created wheel for joeynmt: filename=joeynmt-1.2-py3-none-any.whl size=80910 sha256=4b8bb0da84b3720e17c72f7d57f2121905495edae86d95d479e27b328a58d8b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-0t3hxllr/wheels/e0/75/f9/2caaffdeb17d5a6178b6cff4d645f41aa925964754c873c0b7
Successfully built joeynmt
  Attempting uninstall: joeynmt
    Found existing installation: joeynmt 1.2
    Uninstalling joeynmt-1.2:
      Successfully uninstalled joeynmt-1.2
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [25]:
import torch

In [26]:
torch.__version__

'1.7.1+cu101'

In [27]:
torch.cuda.is_available()

True

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [28]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.sw	     test.en-any.en    train.bpe.en
dev.bpe.en	test.bpe.en  test.en-any.en.1  train.bpe.sw
dev.bpe.sw	test.bpe.sw  test.en-any.en.2  train.en
dev.en		test.en      test.sw	       train.sw
bpe.codes.4000	dev.sw	     test.en-any.en    train.bpe.en
dev.bpe.en	test.bpe.en  test.en-any.en.1  train.bpe.sw
dev.bpe.sw	test.bpe.sw  test.en-any.en.2  train.en
dev.en		test.en      test.sw	       train.sw
BPE Xhosa Sentences
The large shi@@ eld of faith ( See par@@ ag@@ ra@@ ph@@ s 1@@ 2 - 1@@ 4 )
The hel@@ me@@ t of sal@@ v@@ ation ( See par@@ ag@@ ra@@ ph@@ s 1@@ 5 - 1@@ 8 )
I@@ ’@@ ve found that people resp@@ ond well when they see that you are pas@@ sion@@ ate about the Bible and are doing your best to help them . ”
The s@@ word of the spirit ( See par@@ ag@@ ra@@ ph@@ s 1@@ 9 - 2@@ 0 )
With Jehovah’s help , we can stand fir@@ m against him !
Combined BPE Vocab
➌
×@@
Α@@
ז@@
Б@@
ж@@
ю
→@@
м
➋


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [29]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [30]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2021-01-21 13:36:11,677 - INFO - root - Hello! This is Joey-NMT (version 1.2).
/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-01-21 13:36:11,689 - INFO - joeynmt.data - loading training data...
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
2021-01-21 13:36:29,192 - INFO - joeynmt.data - building vocabulary.

In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "$gdrive_path/models/${src}${tgt}_reverse_transformer/"

In [31]:
import torchtext

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [32]:
torchtext.__version__

'0.8.1'

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"